# CNN for Sentence Classification

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['glove-global-vectors-for-word-representation', 'movie-review-sentiment-analysis-kernels-only']


In [4]:
# lets import some stuff
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from keras.models import Model
from keras.layers import *
from keras.utils.np_utils import to_categorical
import re

import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
df = pd.read_csv('../input/movie-review/data.tsv', delimiter='\t')
df = df[['Phrase', 'new_sent']]
df.columns= ["Phrase", "Sentiment"]
pd.set_option('display.max_colwidth', -1)
df.head(3)

,Phrase,Sentiment
0,", this is one adapted - from-television movie that actually looks as if it belongs on the big screen .",1
1,"fails to give his audience a single character worth rooting for -LRB- or worth rooting against , for that matter -RRB-",0
2,A dark comedy that goes for sick and demented humor simply to do so .,0


In [11]:
def clean_str(in_str):
    in_str = str(in_str)
    # replace urls with 'url'
    in_str = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})", "url", in_str)
    in_str = re.sub(r'([^\s\w]|_)+', '', in_str)
    return in_str.strip().lower()


df['text'] = df['Phrase'].apply(clean_str)
data = df

In [12]:
df.Sentiment.value_counts()

1    6000
0    6000
Name: Sentiment, dtype: int64

Now, we need to tokenize our sentences into vectors. Before we do that we need to pick a vector size.

In [15]:
data['l'] = data['Phrase'].apply(lambda x: len(str(x).split(' ')))
print("mean length of sentence: " + str(data.l.mean()))
print("max length of sentence: " + str(data.l.max()))
print("std dev length of sentence: " + str(data.l.std()))

mean length of sentence: 18.754
max length of sentence: 52
std dev length of sentence: 6.937518391335498


In [17]:
# these sentences aren't that long so we may as well use the whole string
sequence_length = 52

In [19]:
max_features = 20000 # this is the number of words we care about

tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
tokenizer.fit_on_texts(data['Phrase'].values)

# this takes our sentences and replaces each word with an integer
X = tokenizer.texts_to_sequences(data['Phrase'].values)

# we then pad the sequences so they're all the same length (sequence_length)
X = pad_sequences(X, sequence_length)

y = pd.get_dummies(data['Sentiment']).values

# where there isn't a test set, Kim keeps back 10% of the data for testing, I'm going to do the same since we have an ok amount to play with
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print("test set size " + str(len(X_test)))

test set size 2400


# Model 1: Glove Embeddings

In [ ]:
embedding_dim = 50
num_filters = 100

inputs = Input(shape=(sequence_length,), dtype='int32')


In [36]:
embeddings_index = {}
f = open(os.path.join('../input/glove-global-vectors-for-word-representation/', 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [37]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 12599 unique tokens.


In [38]:
num_words = min(max_features, len(word_index)) + 1
print(num_words)

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in our tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

12600


In [42]:
inputs_2 = Input(shape=(sequence_length,), dtype='int32')

# note the `trainable=False`, later we will make this layer trainable
embedding_layer_2 = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=False)(inputs_2)

reshape_2 = Reshape((sequence_length, embedding_dim, 1))(embedding_layer_2)

conv_0_2 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)
conv_1_2 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)
conv_2_2 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)

maxpool_0_2 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_2)
maxpool_1_2 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_2)
maxpool_2_2 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_2)

concatenated_tensor_2 = Concatenate(axis=1)([maxpool_0_2, maxpool_1_2, maxpool_2_2])
flatten_2 = Flatten()(concatenated_tensor_2)

dropout_2 = Dropout(0.5)(flatten_2)
output_2 = Dense(units=2, activation='softmax')(dropout_2)

In [43]:
model_2 = Model(inputs=inputs_2, outputs=output_2)
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_2.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 52)           0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 52, 50)       630000      input_5[0][0]                    
__________________________________________________________________________________________________
reshape_5 (Reshape)             (None, 52, 50, 1)    0           embedding_5[0][0]                
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 50, 1, 100)   15100       reshape_5[0][0]                  
__________________________________________________________________________________________________
conv2d_14 

In [44]:
batch_size = 32
history_2 = model_2.fit(X_train, y_train, epochs=30, batch_size=batch_size, verbose=1, validation_split=0.2)

Train on 7680 samples, validate on 1920 samples
Epoch 1/30
7680/7680 [==============================] - 2s 220us/step - loss: 1.1849 - acc: 0.5379 - val_loss: 0.7097 - val_acc: 0.5089
Epoch 2/30
7680/7680 [==============================] - 1s 140us/step - loss: 0.7109 - acc: 0.5305 - val_loss: 0.7086 - val_acc: 0.5115
Epoch 3/30
7680/7680 [==============================] - 1s 184us/step - loss: 0.7094 - acc: 0.5266 - val_loss: 0.7081 - val_acc: 0.4958
Epoch 4/30
7680/7680 [==============================] - 1s 150us/step - loss: 0.7072 - acc: 0.5258 - val_loss: 0.7064 - val_acc: 0.5047
Epoch 5/30
7680/7680 [==============================] - 1s 152us/step - loss: 0.7063 - acc: 0.5215 - val_loss: 0.7047 - val_acc: 0.5073
Epoch 6/30
7680/7680 [==============================] - 1s 150us/step - loss: 0.7027 - acc: 0.5167 - val_loss: 0.7007 - val_acc: 0.5047
Epoch 7/30
7680/7680 [==============================] - 1s 143us/step - loss: 0.6995 - acc: 0.5043 - val_loss: 0.6974 - val_acc: 0.4958


In [46]:
y_hat_2 = model_2.predict(X_test)

In [47]:
accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_2)))

0.4920833333333333

# Model 2: Glove with trainable embeddings

For this model we're going to try the same model again, but this time make the embeddings trainable. That means if during training the model decides on a better embedding for a word then it'll update it

This has the added benifit of updating the words which were randomly assigned a vector (because they weren't in the w2v model)

In [49]:
inputs_3 = Input(shape=(sequence_length,), dtype='int32')
embedding_layer_3 = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=True)(inputs_3)

reshape_3 = Reshape((sequence_length, embedding_dim, 1))(embedding_layer_3)

# note the relu activation
conv_0_3 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)
conv_1_3 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)
conv_2_3 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)

maxpool_0_3 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_3)
maxpool_1_3 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_3)
maxpool_2_3 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_3)

concatenated_tensor_3 = Concatenate(axis=1)([maxpool_0_3, maxpool_1_3, maxpool_2_3])
flatten_3 = Flatten()(concatenated_tensor_3)

dropout_3 = Dropout(0.5)(flatten_3)
output_3 = Dense(units=2, activation='softmax')(dropout_3)

In [50]:
model_3 = Model(inputs=inputs_3, outputs=output_3)
model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_3.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 52)           0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 52, 50)       630000      input_7[0][0]                    
__________________________________________________________________________________________________
reshape_7 (Reshape)             (None, 52, 50, 1)    0           embedding_7[0][0]                
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 50, 1, 100)   15100       reshape_7[0][0]                  
__________________________________________________________________________________________________
conv2d_20 

In [51]:
batch_size = 32
history_3 = model_3.fit(X_train, y_train, epochs=30, batch_size=batch_size, verbose=1, validation_split=0.2)

Train on 7680 samples, validate on 1920 samples
Epoch 1/30
7680/7680 [==============================] - 2s 260us/step - loss: 1.1809 - acc: 0.5400 - val_loss: 0.7157 - val_acc: 0.5047
Epoch 2/30
7680/7680 [==============================] - 1s 164us/step - loss: 0.7202 - acc: 0.5690 - val_loss: 0.7181 - val_acc: 0.7161
Epoch 3/30
7680/7680 [==============================] - 1s 163us/step - loss: 0.7257 - acc: 0.5905 - val_loss: 0.7177 - val_acc: 0.5714
Epoch 4/30
7680/7680 [==============================] - 1s 163us/step - loss: 0.7209 - acc: 0.6758 - val_loss: 0.7554 - val_acc: 0.5625
Epoch 5/30
7680/7680 [==============================] - 1s 189us/step - loss: 0.6896 - acc: 0.7723 - val_loss: 0.6792 - val_acc: 0.7927
Epoch 6/30
7680/7680 [==============================] - 1s 153us/step - loss: 0.6394 - acc: 0.8253 - val_loss: 0.6373 - val_acc: 0.8245
Epoch 7/30
7680/7680 [==============================] - 1s 155us/step - loss: 0.5691 - acc: 0.8602 - val_loss: 0.6061 - val_acc: 0.8396


In [52]:
y_hat_3 = model_3.predict(X_test)

In [53]:
accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_3)))

0.8858333333333334